# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [ ]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [ ]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [ ]:
SELECT * FROM Crop_Group LIMIT 10;

In [ ]:
SELECT * FROM FCID_Description LIMIT 10;

In [ ]:
SELECT * FROM Recipes LIMIT 10;

In [ ]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [ ]:
DROP VIEW IF EXISTS commodity_profile;
CREATE VIEW commodity_profile AS
(SELECT
    i.fcid_code,
    fcid_desc,
    COUNT(DISTINCT seqn) AS popularidade,
    SUM(intake) AS intake_sum,
    (SUM(intake) / COUNT(DISTINCT seqn)) AS intake_avg,
    SUM(intake_bw) / COUNT(DISTINCT seqn) AS intake_avg_bw,
    (SELECT COUNT(DISTINCT food_code)
         FROM recipes AS r
         WHERE r.fcid_code = i.fcid_code
         GROUP BY r.fcid_code
    ) AS recipes
FROM intake i JOIN fcid_description fd ON i.fcid_code = fd.fcid_code
GROUP BY i.fcid_code);
SELECT * FROM commodity_profile LIMIT 10;

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

É esperado, por exemplo, que a média de consumo e a média de consumo por peso sejam proporcionais para todos os alimentos, se não forem, isso pode indicar que alguns alimentos são mais consumidos por pessoas com maior ou menor massa corporal, indicando assim alimentos mais ou menos calóricos
Alimentos com alta soma de consumo porém baixa média podem indicar alimentos nichados, possivelmente indicando algo em comum dentre seus consumidores. Já alimentos populares (em quantidade de consumidores) com baixas somas de consumo podem indicar alimentos pouco significativos na dieta de seus consumidores, como por exemplo temperos, e os com maior soma seriam que compõem maior parte de suas dietas


In [ ]:

DROP VIEW IF EXISTS fcid_intake_ratio;
CREATE VIEW fcid_intake_ratio AS
SELECT
    i.fcid_code,
    fd.fcid_desc AS fcid_description,
    SUM(intake) AS intake_sum,
    COUNT(DISTINCT seqn) AS popularidade,
    SUM(intake) / COUNT(DISTINCT seqn) AS intake_popularity_ratio,
    (SUM(intake_bw) / COUNT(DISTINCT seqn)) AS avg_intake_bw,
    (SUM(intake) / COUNT(DISTINCT seqn)) AS avg_intake,
    (SUM(intake_bw) / COUNT(DISTINCT seqn)) / (SUM(intake) / COUNT(DISTINCT seqn)) AS body_weight_ratio
FROM
    intake i
JOIN
    fcid_description fd ON i.fcid_code = fd.fcid_code
GROUP BY
    i.fcid_code, fd.fcid_desc;

/* Filtrando essa querry por popularidade de forma arbitraria para tentar reduzir o número de outliers */

SELECT fcid_description, intake_sum, popularidade, intake_popularity_ratio FROM fcid_intake_ratio WHERE popularidade > 50 ORDER BY intake_popularity_ratio DESC LIMIT 10;
SELECT fcid_description, avg_intake_bw, avg_intake, body_weight_ratio FROM fcid_intake_ratio WHERE popularidade > 50 ORDER BY body_weight_ratio DESC LIMIT 10


## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?


É esperado, por exemplo, que a média de consumo e a média de consumo por peso sejam proporcionais para todos os alimentos, se não forem, isso pode indicar que alguns alimentos são mais consumidos por pessoas com maior ou menor massa corporal, indicando assim alimentos mais ou menos calóricos
Alimentos com alta soma de consumo porém baixa média podem indicar alimentos nichados, possivelmente indicando algo em comum dentre seus consumidores. Já alimentos populares (em quantidade de consumidores) com baixas somas de consumo podem indicar alimentos pouco significativos na dieta de seus consumidores, como por exemplo temperos, e os com maior soma seriam que compõem maior parte de suas dietas


## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [ ]:
SELECT
    s as seqn,
    ISNULL((SELECT SUM(intake) FROM intake WHERE seqn = s AND cooking_method = 3 GROUP BY seqn) / dt.total, 0) AS razao_fritura
FROM (SELECT seqn s, SUM(intake) AS total FROM intake GROUP BY seqn) AS dt ORDER BY s;